In [8]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import sys
print('Libraries imported.')

Libraries imported.


# Segmenting and Clustering Neighborhoods in Toronto

In [9]:

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [10]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [11]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [14]:
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.648690,-79.385440
1,M2A\n,Not assigned\n,Not assigned\n,43.648690,-79.385440
2,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
3,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
...,...,...,...,...,...
329,M4Z\n,Not assigned\n,Not assigned\n,43.648690,-79.385440
330,M5Z\n,Not assigned\n,Not assigned\n,43.648690,-79.385440
331,M6Z\n,Not assigned\n,Not assigned\n,43.648690,-79.385440
332,M7Z\n,Not assigned\n,Not assigned\n,43.648690,-79.385440


In [15]:

neighbors = neighbors[neighbors.Borough != 'Not assigned\n']


In [16]:
 for i in neighbors['Neighborhood']:
        if i == 'Not assigned\n':
            neighbors['Neighborhood'] = neighbors['Borough']

In [17]:
neighbors = neighbors.replace(r'\n',  ' ', regex=True)

In [18]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.752935,-79.335641
3,M4A,North York,Victoria Village,43.728102,-79.311890
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
315,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
320,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
323,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
324,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


In [56]:
downtown_toronto_data = neighbors[neighbors['Borough'] == 'Downtown Toronto ']
# eliminate 'Postalcode' column
downtown_toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
22,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
31,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855


In [32]:
CLIENT_ID = 'GYLFCYRMJHYKEGH3AT1K00UZORNDQ3PAHMKRCLV5OLU1DZGV' # your Foursquare ID
CLIENT_SECRET = 'PZWJGRAMCNXIXNALQZCD510PXOQB0MCA1A2WMUCYPKXZIH3K' # your Foursquare Secret
VERSION = '20200602' # Foursquare API version
limit=20
from geopy.geocoders import Nominatim

In [33]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

C:\Users\joaof\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [36]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [37]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

# End of "Segmenting and Clustering Neighborhoods in Toronto"

In [38]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Regent Park, Harbourfront 
Queen's Park, Ontario Provincial Government 
Garden District, Ryerson 
St. James Town 
Berczy Park 
Central Bay Street 
Christie 
Richmond, Adelaide, King 
Harbourfront East, Union Station, Toronto Islands 
Toronto Dominion Centre, Design Exchange 
Commerce Court, Victoria Hotel 
University of Toronto, Harbord 
Kensington Market, Chinatown, Grange Park 
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport 
Rosedale 
Stn A PO Boxes 
Regent Park, Harbourfront 
Queen's Park, Ontario Provincial Government 
Garden District, Ryerson 
St. James Town 
Berczy Park 
Central Bay Street 
Christie 
Richmond, Adelaide, King 
Harbourfront East, Union Station, Toronto Islands 
Toronto Dominion Centre, Design Exchange 
Commerce Court, Victoria Hotel 
University of Toronto, Harbord 
Kensington Market, Chinatown, Grange Park 
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island a

In [43]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(624, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop


In [132]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,40,40,40,40,40,40
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",40,40,40,40,40,40
Central Bay Street,40,40,40,40,40,40
Christie,24,24,24,24,24,24
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",40,40,40,40,40,40
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",40,40,40,40,40,40
"Harbourfront East, Union Station, Toronto Islands",12,12,12,12,12,12


In [45]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 121 uniques categories.


In [46]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,...,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [48]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park ----
            venue  freq
0    Cocktail Bar  0.10
1            Café  0.05
2  Breakfast Spot  0.05
3    Concert Hall  0.05
4          Museum  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport ----
                  venue  freq
0    Italian Restaurant  0.10
1                  Park  0.10
2  Gym / Fitness Center  0.05
3     French Restaurant  0.05
4   Peruvian Restaurant  0.05


----Central Bay Street ----
                       venue  freq
0                Coffee Shop  0.15
1        Japanese Restaurant  0.10
2                      Hotel  0.05
3         Chinese Restaurant  0.05
4  Middle Eastern Restaurant  0.05


----Christie ----
           venue  freq
0  Grocery Store  0.25
1           Café  0.25
2     Baby Store  0.08
3           Park  0.08
4    Coffee Shop  0.08


----Church and Wellesley ----
            venue  freq
0     Coffee Shop  0.10
1       Wine Shop  0.05
2  Breakfast Spot  0.05
3          

In [49]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Park,Tailor Shop,Concert Hall,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum
1,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Italian Restaurant,Garden,Speakeasy,Mexican Restaurant,New American Restaurant,Market,Peruvian Restaurant,Caribbean Restaurant,Ramen Restaurant
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Hotel,Modern European Restaurant,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Dessert Shop,Ramen Restaurant,Bubble Tea Shop
3,Christie,Grocery Store,Café,Coffee Shop,Athletics & Sports,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub
4,Church and Wellesley,Coffee Shop,Wine Shop,Bookstore,Flower Shop,Ice Cream Shop,Juice Bar,Dance Studio,Men's Store,Pizza Place,Café
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Japanese Restaurant,Tea Room,Restaurant,Museum,Ice Cream Shop,Hotel,Tailor Shop,Bakery
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gluten-free Restaurant,Bakery,Pizza Place,Pub,Japanese Restaurant,Seafood Restaurant,Bookstore
7,"Garden District, Ryerson",Café,Ramen Restaurant,Sandwich Place,Hotel,Music Venue,Clothing Store,Comic Shop,Lounge,Pizza Place,Plaza
8,"Harbourfront East, Union Station, Toronto Isla...",Harbor / Marina,Farm,Fast Food Restaurant,Theme Park,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Cocktail Bar,Bakery,Organic Grocery,Coffee Shop,Mexican Restaurant,Cheese Shop,Caribbean Restaurant,Dessert Shop


In [126]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18] 

array([1, 1, 0, 2, 0, 0, 0, 1, 3, 0, 0, 0, 0, 4, 0, 1, 1, 0])

In [133]:
downtown_toronto_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels
0,Berczy Park,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,...,0.05,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.00,1
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,1
2,Central Bay Street,0.0,0.0,0.0,0.05,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0
3,Christie,0.0,0.0,0.0,0.00,0.0,0.0,0.083333,0.0,0.083333,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,2
4,Church and Wellesley,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,0.0,0.05,0.0,0.00,0.0,0.0,0.05,0


In [141]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = neighborhoods_venues_sorted

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged =  downtown_toronto_merged.join(downtown_toronto_venues.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,Cocktail Bar,Park,Tailor Shop,Concert Hall,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,Cocktail Bar,Park,Tailor Shop,Concert Hall,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,Cocktail Bar,Park,Tailor Shop,Concert Hall,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,Cocktail Bar,Park,Tailor Shop,Concert Hall,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,Cocktail Bar,Park,Tailor Shop,Concert Hall,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [142]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Venue Latitude'], downtown_toronto_merged['Venue Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [143]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Dessert Shop,Ramen Restaurant,Bubble Tea Shop,0,43.656072,-79.385653,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
2,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Dessert Shop,Ramen Restaurant,Bubble Tea Shop,0,43.656072,-79.385653,The Elm Tree Restaurant,43.657397,-79.383761,Modern European Restaurant
2,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Dessert Shop,Ramen Restaurant,Bubble Tea Shop,0,43.656072,-79.385653,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Dessert Shop,Ramen Restaurant,Bubble Tea Shop,0,43.656072,-79.385653,Rolltation,43.654918,-79.387424,Japanese Restaurant
2,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Chinese Restaurant,Dessert Shop,Ramen Restaurant,Bubble Tea Shop,0,43.656072,-79.385653,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,Coffee Shop,Pizza Place,Deli / Bodega,Pub,Restaurant,Japanese Restaurant,Hotel,0,43.646923,-79.381626,Sam James Coffee Bar (SJCB),43.647881,-79.384332,Café
17,Coffee Shop,Pizza Place,Deli / Bodega,Pub,Restaurant,Japanese Restaurant,Hotel,0,43.646923,-79.381626,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
17,Coffee Shop,Pizza Place,Deli / Bodega,Pub,Restaurant,Japanese Restaurant,Hotel,0,43.646923,-79.381626,Petit Four Bakery,43.647744,-79.379588,Sandwich Place
17,Coffee Shop,Pizza Place,Deli / Bodega,Pub,Restaurant,Japanese Restaurant,Hotel,0,43.646923,-79.381626,Olly Fresco's,43.646912,-79.379597,Deli / Bodega


In [144]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cocktail Bar,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Cocktail Bar,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Cocktail Bar,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Cocktail Bar,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,LCBO,43.642944,-79.372440,Liquor Store
0,Cocktail Bar,Liquor Store,Café,Jazz Club,Restaurant,Breakfast Spot,Museum,1,43.645196,-79.373855,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,Park,Bookstore,Yoga Studio,Bakery,College Arts Building,Dessert Shop,Comfort Food Restaurant,1,43.663281,-79.398088,FLOCK Rotisserie + Greens,43.662637,-79.403798,Comfort Food Restaurant
18,Park,Bookstore,Yoga Studio,Bakery,College Arts Building,Dessert Shop,Comfort Food Restaurant,1,43.663281,-79.398088,Kimpton St. George,43.667368,-79.400627,Hotel
18,Park,Bookstore,Yoga Studio,Bakery,College Arts Building,Dessert Shop,Comfort Food Restaurant,1,43.663281,-79.398088,Innis Town Hall,43.665420,-79.399546,College Arts Building
18,Park,Bookstore,Yoga Studio,Bakery,College Arts Building,Dessert Shop,Comfort Food Restaurant,1,43.663281,-79.398088,Bar Mercurio,43.667428,-79.400330,Restaurant


In [145]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]


,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Fiesta Farms,43.668471,-79.420485,Grocery Store
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Scout and Cash Caffe,43.667360,-79.419938,Café
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Contra Cafe,43.669107,-79.426105,Café
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Christie Pits Park,43.664177,-79.420466,Park
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Starbucks,43.671530,-79.421400,Coffee Shop
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Vermont Square Park,43.670493,-79.415399,Playground
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Karma Co-operative,43.668185,-79.414504,Grocery Store
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Faema Caffe,43.671046,-79.419297,Café
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Stubbe Chocolates,43.671566,-79.421289,Candy Store
3,Grocery Store,Playground,Candy Store,Baby Store,Park,Fried Chicken Joint,Gastropub,2,43.668602,-79.420387,Loblaws,43.671657,-79.421364,Grocery Store


In [146]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,Far Enough Farm,43.619853,-79.371797,Farm
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,RCYC South Island,43.622391,-79.369388,Harbor / Marina
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,Royal Canadian Yacht Club,43.623886,-79.370560,Harbor / Marina
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,BeaverTails,43.623538,-79.367601,Fast Food Restaurant
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,Snake Island,43.624590,-79.363982,Park
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,Centreville Railroad Station,43.620826,-79.373451,Theme Park
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,Far Enough Farm,43.619853,-79.371797,Farm
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,RCYC South Island,43.622391,-79.369388,Harbor / Marina
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,Royal Canadian Yacht Club,43.623886,-79.370560,Harbor / Marina
8,Harbor / Marina,Park,Garden,Gastropub,College Gym,College Rec Center,Comfort Food Restaurant,3,43.62375,-79.3692,BeaverTails,43.623538,-79.367601,Fast Food Restaurant


In [148]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Summerhill Market,43.686265,-79.375458,Grocery Store
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Rosedale Park,43.682328,-79.378934,Playground
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Whitney Park,43.682036,-79.373788,Park
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Scoops Convenience Boutique,43.686148,-79.375828,Candy Store
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Summerhill Market,43.686265,-79.375458,Grocery Store
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Rosedale Park,43.682328,-79.378934,Playground
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Whitney Park,43.682036,-79.373788,Park
13,Grocery Store,Wine Shop,Discount Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,4,43.681893,-79.376706,Scoops Convenience Boutique,43.686148,-79.375828,Candy Store
